In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


### Make scene

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 3, -2, 2, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [4]:
from pkg.geometry.geometry import *
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
BRUSH_NAME = "brush_face"
brush_face = add_indy_sweep_tool(gscene, ROBOT_NAME, face_name=BRUSH_NAME, tool_offset=0.12, color_col=(0.8,0.8,0.8,0.2))
brush_b = pscene.create_binder(BRUSH_NAME, BRUSH_NAME, WayFramer, point=(0,0,brush_face.dims[2]/2), rpy=(0,np.pi,0))
# from demo_utils.environment import add_kiro_indytool_down
# add_kiro_indytool_down(gscene, zoff=0, tool_link="indy1_tcp", face_name="brush_face", ext_off=0.032, tool_dim=(0.08, 0.32))
gscene.show_pose(crob.home_pose)

MOBILE_PUSH_NAME = "mpush"
MOBILE_PUSH_HEIGHT = 0.4
MOBILE_PUSH_KEY = 1
front_dist = 0.45+CLEARANCE
mpush_g = gscene.create_safe(GEOTYPE.SPHERE, MOBILE_PUSH_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, MOBILE_PUSH_HEIGHT), rpy=(0,np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5))
mpush_b = pscene.create_binder(MOBILE_PUSH_NAME, MOBILE_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=MOBILE_PUSH_KEY)

floor_b = pscene.create_binder("floor_ws", "floor_ws", PlaceFrame, point=(0,0,WALL_THICKNESS/2), key=MOBILE_PUSH_KEY)

TOOL_PUSH_NAME = "tpush"
TOOL_PUSH_HEIGHT = 0.03
TOOL_PUSH_KEY = 10
brush_col = gscene.NAME_DICT[BRUSH_NAME+"_col"]
TOOL_DIM = brush_col.dims
tpush_g = gscene.create_safe(GEOTYPE.SPHERE, TOOL_PUSH_NAME, link_name=TIP_LINK,dims=(0.01,0.01,0.01),
                           center=(-TOOL_DIM[0]/2-CLEARANCE, 0, 0), rpy=(0,-np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5), parent=BRUSH_NAME)
tpush_b = pscene.create_binder(TOOL_PUSH_NAME, TOOL_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=TOOL_PUSH_KEY)


## Table

In [6]:
TABLE_DIM = (0.6, 1.2, 0.7)
TABLE_NAME = "table"
TABLE_LOC = (2, 0, TABLE_DIM[2]/2)
TABLE_RPY = (0,0,0)

In [7]:
table_g = gscene.create_safe(GEOTYPE.BOX, TABLE_NAME, link_name="base_link", dims=TABLE_DIM,
                             center=TABLE_LOC, rpy=TABLE_RPY,
                             fixed=True, collision=True, color=(0.8,0.8,0.8,1))
table_b = pscene.create_binder(TABLE_NAME, TABLE_NAME, PlaceFrame, point=(0,0,TABLE_DIM[2]/2), key=TOOL_PUSH_KEY)

## Waypoints

In [8]:
WP_NAME = "wp_{}"
WP_DIM = (0.1,0.1,0.01)
WP_LOC = (-0.2, 0.0, TABLE_DIM[2]/2+WP_DIM[2]/2)
WP_RPY = (0,0,0)

In [9]:
wp_g = gscene.create_safe(GEOTYPE.BOX, WP_NAME.format(0), link_name="base_link", dims=WP_DIM,
                          center=WP_LOC, rpy=WP_RPY,
                          fixed=True, collision=True, color=(0.0,0.0,0.8,1), parent=TABLE_NAME)
waypoint_s = pscene.create_subject(oname="waypoints", gname=TABLE_NAME, _type=WaypointTask,
                                 action_points_dict={
                                     wp_g.name: WayFrame(
                                         wp_g.name, wp_g, [0, 0, wp_g.dims[2] / 2], [0, 0, 0])})

## Clock

In [10]:
CLOCK_NAME = "clock"
CLOCK_DIM = (0.14,0.05,0.076)
clock_center = np.add(wp_g.center[:2]+(0,), (0,0.1,TABLE_DIM[2]+CLOCK_DIM[2]/2+CLEARANCE))

In [11]:
clock_g = gscene.create_safe(GEOTYPE.BOX, CLOCK_NAME, link_name="base_link",dims=CLOCK_DIM,
                           center=clock_center, rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.5,0.3,1))

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(CLOCK_DIM, -R[:,2])/2, 4)
    point[2] = -CLOCK_DIM[2]/2 + TOOL_PUSH_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, clock_g, point=point, rpy=Rot2rpy(R), key=TOOL_PUSH_KEY))

slide_point = SlidePoint("bottom_p", clock_g, point=(0,0,-CLOCK_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                         binded_on=table_b, dist_push=0.2, dir_push=[-1,0,0], key=TOOL_PUSH_KEY)

clock_s = pscene.create_subject(oname=CLOCK_NAME, gname=CLOCK_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

## Bag

In [12]:
BAG_NAME = "bag"
BAG_DIM = (0.4, 0.29,0.64)

In [13]:
bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME, link_name="base_link",dims=BAG_DIM,
                           center=(1,0,BAG_DIM[2]/2), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.8,0.8,1))
handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                           center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME)

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
    point[2] = -BAG_DIM[2]/2 + MOBILE_PUSH_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=MOBILE_PUSH_KEY))

slide_point = SlidePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                                           binded_on=floor_b, dist_push=1.0, key=MOBILE_PUSH_KEY)

bag_s = pscene.create_subject(oname=BAG_NAME, gname=BAG_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

### prepare planner

In [14]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=100, allow_joint_motion=True)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
mplan.motion_filters = [gcheck, rcheck]
mplan.incremental_constraint_motion = True

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[WARN] reach data is not ready for kmb. Ignoring this robot...


### Custom rules

In [15]:
# joint motion is not allowed when pushing object with holder
idc_push = np.where([stype == PushObject for stype in pscene.subject_type_list])[0]
tplan.explicit_rule = lambda pscene, node, leaf: not any([node[idx]==leaf[idx] and "push" in node[idx] for idx in idc_push])

In [16]:
# generate base pose to reach targets
N_samples = 50
thetas_available = np.array([np.pi/2*i for i in range(4)])

xy_list = floor_b.geometry.get_vertice_radius_from(crob.home_dict)[0][:,:2]
xy_max = np.max(xy_list,axis=0)
xy_min = np.min(xy_list,axis=0)

targets = [pscene.subject_dict[sname] for sname, stype in zip(pscene.subject_name_list, pscene.subject_type_list) if stype == WaypointTask]
wps_dict = {target.oname: [target.action_points_dict[wname] for wname in target.action_points_order] for target in targets}
def gen_reaching_base(state):
    xyt_samples = np.array([np.random.uniform(xy_min, xy_max).tolist()
                            +[np.random.choice(thetas_available)] 
                            for _ in range(N_samples)])

    # check availability for wps
    Twe = np.linalg.inv(brush_b.Toff_lh)
    Tbe_mat = []
    for target in targets:
        i_s = pscene.subject_name_list.index(target.oname)
        ntem = state.node[i_s]
        wps = wps_dict[target.oname][ntem:] # exclude already reached waypoints
        Tbe_arr = []
        for wp in wps:
            Tbe = np.matmul(wp.get_tf_handle(crob.home_pose), Twe)
            Tbe_arr.append(Tbe)
        Tbe_mat.append(Tbe_arr)
        
    # score with more weights on reachable num on same targets
    score_mat = []
    for i_x, xyt in enumerate(xyt_samples):
        Tbr = gscene.get_tf(ROBOT_BASE, [xyt[0], xyt[1], xyt[2]]+[0]*9)
        score_list = []
        for Tbe_arr in Tbe_mat: # iterate targets
            score_t = 0
            for Tbe in Tbe_arr: # iterate waypoints
                Tre = np.matmul(np.linalg.inv(Tbr), Tbe)
                res = rcheck.check_T(ROBOT_NAME, Tre)
                if res:
                    Q = np.copy(state.Q)
                    Q[:3] = xyt
                    res = mplan.validate_trajectory([Q])
                    score_t += res
            score_list.append(score_t)
        score_mat.append(score_list)
    score_mat = np.square(score_mat) # (pose_num, target_num), square to weight more on large counts
    score_list = np.sum(score_mat, axis=-1,dtype=float)
    score_sum = np.sum(score_list)
    if score_sum>0:
        score_list = np.divide(score_list, score_sum)
        i_select = np.random.choice(range(N_samples), p=score_list)
        Q = np.copy(state.Q)
        Q[:3] = xyt_samples[i_select]
        return Q
    return None

tplan.config_gen = gen_reaching_base

### Plan

In [17]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
gscene.update_markers_all()
print(pscene.subject_name_list)
print(initial_state.node)

['bag', 'clock', 'waypoints']
('floor_ws', 'table', 0)


In [21]:
goal_nodes = [('floor_ws', 'table', 1)]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=False, add_homing=True)

try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 0->1 (0.08/100.0 s, steps/err: 2(5.25999069214 ms)/0)
try: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching

end
constrained motion tried: True
result: 11 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 11->12 (1.94/100.0 s, steps/err: 64(238.200902939 ms)/5.05913003119e-08)
try: 12 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 12 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 10 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 10 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 10->13 (1.99/100.0 s, steps/err: 2(2.63094902039 ms)/0)
try: 13 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 13 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 12 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 12 - 

transition motion tried: True
result: 1 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 1->20 (3.68/100.0 s, steps/err: 17(198.161125183 ms)/0.0020186160312)
try: 20 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
result: 20 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 0 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 13 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
result: 13 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 2 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 2 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 8 -

try: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 6->27 (6.13/100.0 s, steps/err: 2(3.10111045837 ms)/0)
try: 27 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 27 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 15 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 15 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 22 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 22 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 19 - ('floor_ws', 'table', 0)->

transition motion tried: True
result: 31 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 31->36 (7.67/100.0 s, steps/err: 23(319.787025452 ms)/0.00138960230165)
try: 36 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
result: 36 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 30 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 30 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 8 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
result: 8 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 35 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 35 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 28 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 28 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 28->37 (7.

...

result: 27 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 41 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
result: 41 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 66 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 66 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 69 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
result: 69 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
result: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 61 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 61 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 68 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 68 - ('floor_ws', 'table', 0)->('floo

result: 19 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 18 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 18 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 18->80 (20.4/100.0 s, steps/err: 2(2.56204605103 ms)/0)
try: 80 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 80 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 50 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
result: 50 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 32 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
error: 0.35
try transition motion
transition motion tried: True
result: 32 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 32->81 (20.57/100.0 s, steps/err: 3(164.701938629 ms)/0.00181310527474)
try: 81 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
error: 0.42
end
constrained motion tried: True
result: 81 - ('mpush'

end
constrained motion tried: True
result: 11 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 11->91 (22.7/100.0 s, steps/err: 23(129.610061646 ms)/3.3672170614e-08)
try: 91 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 91 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 78 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
result: 78 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 67 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
error: 0.49
end
constrained motion tried: True
result: 67 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 67->92 (22.92/100.0 s, steps/err: 50(205.742835999 ms)/5.19596669202e-08)
try: 92 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 92 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 38 - ('mpush', 'table', 0)->('floor_ws', 'table', 

Motion Plan Failure
result: 6 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 59 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 59 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 45 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
error: 0.0015
try transition motion
transition motion tried: True
result: 45 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 45->101 (27.73/100.0 s, steps/err: 2(385.318040848 ms)/0.00154153218292)
try: 101 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
error: 0.12
end
constrained motion tried: True
result: 101 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 101->102 (27.81/100.0 s, steps/err: 13(75.3660202026 ms)/9.85714032818e-08)
try: 102 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 102 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 

result: 73 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
======================= terminated 0: node queue empty 1th  (31.7/100.0) ===============================
========================== FINISHED (31.7 / 100.0 s) ==============================]
generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
generate table - PlanConditions
generate table - PlanList
button clicked
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
button action done
button clicked
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 

In [19]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [20]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

IndexError: list index out of range

generate table - Handlegenerate table - Geometry

generate table - Binder
generate table - Object
generate table - PlanConditions
generate table - PlanList
button clicked
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
button action done
generate table - Handle
generate table - Bindergenerate table - Geometry

generate table - Object
generate table - PlanConditions
generate table - PlanList
button clicked
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
('floor_ws', 'tpush', 0)->('floor_ws', 'table', 0)
button action done
button clicked
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
button action done
button clicked
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_